In [ ]:
# import libraries
import io
import os
import sys
import uuid
import pickle

import google.generativeai as genai
import gradio as gr
import numpy as np

from PIL import Image
from dotenv import load_dotenv

from tensorflow import keras
from tensorflow.keras.preprocessing import image

sys.path.append(os.path.abspath(r"./utilities"))
import MovieScriptGenerator
import MoviePosterGenerator

In [2]:
# temporary directory to save images that the user uploads
SAVE_DIR = "temp_images"
os.makedirs(SAVE_DIR, exist_ok=True)

In [3]:
def getName(class_id):
    if class_id == 0:
        return "Buildings"
    elif class_id == 1:
        return "Forest"
    elif class_id == 2:
        return "Glacier"
    elif class_id == 3:
        return "Mountain"
    elif class_id == 4:
        return "Sea"
    elif class_id == 5:
        return "Street"
    else:
        return "Unknown"

In [4]:
def run_prediction(filePath):
    print("Running prediction")
    #model = keras.models.load_model("../models/keras_model.h5")
    #model = keras.models.load_model("../models/model_softmax_ohe_10e.keras")
    #model = keras.models.load_model("../models/tuner_model.keras")
    model = keras.models.load_model("../models/tuned_model_randomsearch.keras")

    img = Image.open(filePath)
    img = img.resize((150, 150))

    img_array = image.img_to_array(img)
    img_array = img_array / 255.0 
    img_array = np.expand_dims(img_array, axis=0)

    shape = img_array.shape
    dtype = (img_array.dtype)

    predictions = model.predict(img_array)
    confidence = np.max(predictions)
    predicted_class = np.argmax(predictions)
    return f"""AAA image classification identifies the provided image to be {getName(predicted_class)} with {(confidence*100):.2f}% confidence"""

In [5]:
def generate(image):
    """
    Processes an input image to generate a movie script and a resized movie poster.

    Args:
        image (PIL.Image.Image): The input image.

    Returns:
        tuple: (str, PIL.Image.Image)
            - The generated movie script as a string.
            - The resized movie poster as a PIL.Image.Image object.
    """
    file_path = os.path.join(SAVE_DIR, f"{uuid.uuid4().hex}.jpg")
    image.save(file_path)
    movieScriptGenerator = MovieScriptGenerator.MovieScriptGenerator()
    script = movieScriptGenerator.generate_script(file_path)
    moviePosterGenerator = MoviePosterGenerator.MoviePosterGenerator()
    poster = moviePosterGenerator.generate_poster(script)
    new_size = (300, 300)
    resized_poster = poster.resize(new_size, Image.LANCZOS)
    predict = run_prediction(file_path)
    return predict, script, resized_poster

In [ ]:
# Creates a Gradio interface for CNN processing, generating a movie script and poster from an image

with gr.Blocks(theme=gr.themes.Soft()) as gui:
    # 🎬 Title
    gr.Markdown("## 🎥 Movie Script & Poster Generator 🎭")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Upload an Image 🖼")
            image_input = gr.Image(type="pil", label="Paste or Upload Image")  
            generate_button = gr.Button("🤖 Process the image 🤖") 

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 📊 Predicted Information")
            text_predict = gr.Textbox(label="Prediction Information", interactive=False)
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### 📜 Generated Movie Script")
            text_output = gr.Markdown(label="Movie Information")
        
        with gr.Column():
            gr.Markdown("### 🎨 Generated Movie Poster")
            image_output = gr.Image(type="pil", label="Movie Poster")

    generate_button.click(fn=generate, inputs=image_input, outputs=[text_predict, text_output, image_output])

# Launch the UI
gui.launch(share=True)